# 103090 - Big Data Project


## Activity 2: Results of the Big Data Project

In [ ]:
!pip install pyspark

In [ ]:
!pip install matplotlib

In [ ]:
!pip install seaborn

In [ ]:
!pip install pandas

In [ ]:
import pyspark.sql
import math

SPARK_ENDPOINT = "local[*]"
sparkSession = pyspark.sql.SparkSession.builder.master(SPARK_ENDPOINT)\
    .config("spark.executor.memory", "16g")\
    .config("spark.driver.memory", "16g")\
    .config("spark.driver.maxResultSize", "4096m").getOrCreate()
sparkContext = sparkSession.sparkContext
sparkSession

In [ ]:
moviesDataFrame = sparkSession.read.json("datasets/yelp_academic_dataset_review.json")
moviesDataFrame.show()
df_reviews = moviesDataFrame.toPandas()

In [ ]:
df_reviews.head()
df_reviews = df_reviews.drop(['_corrupt_record','cool','date','funny','review_id','text','useful','user_id'], axis=1)

In [ ]:
df_reviews.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# file_path_business = "datasets/yelp_academic_dataset_business.json"
file_path_review = "datasets/yelp_academic_dataset_review.json"
file_path_usa = "datasets/usa_state_2022.csv"

# df = pd.read_json(file_path_business)
df_review = pd.read_json(file_path_review,lines=True)
df_usa = pd.read_csv(file_path_usa, delimiter=',')


First look to the dataset

In [ ]:
# df.info()
df_usa.info()
df_review.info()

#### We sum the total hours a business is open per week

In [ ]:

def parse_time(time_str):
    """
    Parse a time string in the format 'H:M' to minutes.
    """
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def calculate_business_hours(hours_data):
    """
    Calculate the total weekly business hours from the hours data.
    """

    if hours_data is None:
        return 0

    total_minutes = 0
    for day, hours in hours_data.items():
        if '-' in hours:  # Ensure the business is open that day
            open_time, close_time = hours.split('-')
            total_minutes += parse_time(close_time) - parse_time(open_time)
    return total_minutes / 60  # Convert total minutes to hours

df['hours'] = df['hours'].apply(calculate_business_hours)

#### We calculate the total number of categories to which each business belongs

In [ ]:
def count_categories(category_string):
    """
    Count the number of categories in the category string.
    """
    if pd.isna(category_string):
        return 0  # Return 0 if the category string is NaN
    categories = category_string.split(', ')
    return len(categories)

# Apply the function to create a new column
df['category_count'] = df['categories'].apply(count_categories)
# Filtrar utilizando loc
test = df.loc[df['category_count'] > 20]

# Imprimir el DataFrame filtrado
print(test)

In [ ]:
categorical = df.select_dtypes(include = "object")
continuous = df.select_dtypes(include = "number")

In [ ]:
categorical.info()

In [ ]:
categorical

In [ ]:
continuous.info()

In [ ]:
continuous

#### Data Visualization: Understanding distributions and relationships

In [ ]:
sns.countplot(x='stars', data=df)
plt.title('Distribution of Business Ratings')
plt.show()

In [ ]:
average_stars = df['stars'].mean()
print(average_stars)
# Plotting the average stars
plt.figure(figsize=(8, 6))
plt.bar('Average Stars', average_stars, color='green')
plt.ylabel('Stars')
plt.title('Average Stars of All Businesses')
plt.ylim(0, 5)  # Setting y-axis limit to match star rating scale
plt.show()

In [ ]:
business_per_state = df['state'].value_counts()
# Geographical Distribution
plt.figure(figsize=(8, 6))
sns.barplot(x=business_per_state.index[:10], y=business_per_state.values[:10], palette='viridis')
plt.title('Top 10 States by Business Count')
plt.xlabel('State')
plt.ylabel('Number of Businesses')

In [ ]:
avg_reviews = df['review_count'].mean()
print(avg_reviews)
# Plot for Average Reviews
plt.figure(figsize=(8, 6))
sns.barplot(x=['Average Reviews'], y=[avg_reviews], palette=['green'])
plt.title('Average Review Count')
plt.ylabel('Average Reviews')

In [ ]:
df['categories'] = df['categories'].fillna('')
all_categories = [category for sublist in df['categories'].str.split(', ') for category in sublist if category]
categories_count = pd.Series(all_categories).value_counts().head(10)
# Top Business Categories
plt.figure(figsize=(8, 6))
sns.barplot(x=categories_count.values, y=categories_count.index, palette='rocket')
plt.title('Top 10 Business Categories')
plt.xlabel('Count')
plt.ylabel('Category')

In [ ]:
# Function to detect outliers using IQR
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))).sum()
    return outliers
df_new = df
# Detecting outliers in 'review_count'
outliers_reviews = detect_outliers(df_new, 'review_count')
print(outliers_reviews)

# Data Quality Assessment: Checking for missing values
missing_values = df_new.isnull().mean().sort_values(ascending=False).head(10)

# Creating the dashboard
plt.figure(figsize=(15, 6))

# Plot for Outliers
plt.subplot(1, 2, 1)
sns.barplot(x=['Review Count'], y=[outliers_reviews], palette='coolwarm')
plt.title('Number of Outliers in Review Count')
plt.ylabel('Number of Outliers')

plt.tight_layout()
plt.show()

# Returning text summary of potential data problems
potential_problems = {
    'Outliers': {
        'Review Count': outliers_reviews
    },
    'Top Missing Values Columns': missing_values.to_dict()
}

potential_problems

In [ ]:
# We will create a box plot for 'stars' and 'review_count' to identify any outliers in these two fields
plt.figure(figsize=(10, 6))

# Creating a box plot
plt.boxplot( df['review_count'], labels=[ 'Review Count'])
plt.title('Box Plot Review Count')
plt.ylabel('Values')

# Displaying the plot
plt.show()

In [ ]:
for column in df:
  if df[column].isnull().sum() > 0:
    print("Column " + df[column].name + " has " + str(df[column].isnull().sum()) + " missing values.")

In [ ]:
df['high_quality'] = (df['review_count'] >= 100).astype(int)

# Setting up a single plot instead of a subplot grid
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['high_quality'], y=df['stars'])

plt.title('Box Plot of Stars by High Quality (Review Count >= 100)')
plt.xlabel('High Quality (0 = No, 1 = Yes)')
plt.ylabel('Stars')

# Displaying the plot
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['high_quality'], y=df['hours'])

plt.title('Box Plot of hours by review_count (Review Count >= 100)')
plt.xlabel('High Quality (0 = No, 1 = Yes)')
plt.ylabel('Hours')

# Displaying the plot
plt.show()

In [ ]:
df['high_quality'] = (df['stars'] >= 4.5).astype(int)
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['high_quality'], y=df['hours'])

plt.title('Box Plot of Hours by Stars (stars >= 4.5)')
plt.xlabel('Stars (0 = No, 1 = Yes)')
plt.ylabel('Hours')

# Displaying the plot
plt.show()

In [ ]:
df['high_quality'] = (df['review_count'] >= 100).astype(int)
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['high_quality'], y=df['category_count'])

plt.title('Box Plot of Stars by review_count (Review Count >= 100)')
plt.xlabel('High Quality (0 = No, 1 = Yes)')
plt.ylabel('category_count')

# Displaying the plot
plt.show()

In [ ]:
df['high_quality'] = (df['stars'] >= 4.5).astype(int)
plt.figure(figsize=(8, 6))
sns.boxplot(x=df['high_quality'], y=df['category_count'])

plt.title('Box Plot of category_count by Stars (stars >= 4.5)')
plt.xlabel('Stars (0 = No, 1 = Yes)')
plt.ylabel('category_count')

# Displaying the plot
plt.show()

Next steps:
* Select the features that we want to use and discard the others.
* Impute the missing values using different imputers.
* Encode the values to work with an specific ml model.
* Filter by categories/attributes
* Create new categories, make the data set more narrow

#### USA state population: https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html

#### Average salary state: https://www.sofi.com/learn/content/average-salary-in-us/

#### Dataset: https://www.yelp.com/dataset